In [1]:
import pandas as pd
import networkx as nx

In [11]:
df = pd.read_csv("trade_data_2000s.csv", index_col=0)

In [12]:
df

,location_id,partner_id,product_id,year,export_value,import_value,sitc_eci,sitc_coi,location_code,partner_code,sitc_product_code
945000,3,6,1352,2003.0,0.0,4281.0,1.535351,0.268432,AIA,ANT,8423
945001,25,6,1352,2003.0,0.0,26220.0,0.308971,-0.561269,BHS,ANT,8423
945002,29,6,1352,2003.0,0.0,84921.0,-0.546138,-0.787781,BLZ,ANT,8423
945003,32,6,1352,2003.0,8011.0,0.0,0.636739,1.422593,BRA,ANT,8423
945004,33,6,1352,2003.0,0.0,2387.0,0.499073,-0.065065,BRB,ANT,8423
...,...,...,...,...,...,...,...,...,...,...,...
3982467,166,242,1363,2019.0,2379.0,0.0,0.099708,0.079208,NZL,WSM,8443
3982468,127,210,1363,2019.0,0.0,2392.0,0.263772,-0.709830,LCA,SXM,8443
3982469,231,210,1363,2019.0,11067.0,0.0,1.528196,0.297825,USA,SXM,8443
3982470,77,27,1363,2019.0,1706.0,0.0,1.345057,1.075400,FRA,BLM,8443


In [7]:
df_grouped = df.groupby(['location_id', 'partner_id']).sum()
df_grouped = df_grouped.drop(['sitc_eci', 'sitc_coi','location_code','partner_code','sitc_product_code'], axis=1)

In [8]:
df_grouped

product_id      year  export_value  import_value
location_id partner_id                                                  
0           6               182754  269006.0     4321927.0     9563522.0
            7                10915   16144.0        3340.0       23713.0
            8                64105   94412.0           0.0      405164.0
            13                2712    4013.0        1444.0       11742.0
            15               13624   20111.0        8528.0       49917.0
...                            ...       ...           ...           ...
250         242               1373    2001.0       35725.0           0.0
            243             308236  454397.0    32896906.0     1327938.0
            246             577966  852067.0   380439090.0     6332941.0
            248             111737  164994.0      415160.0     3294058.0
            249             523829  771434.0    17995633.0     7816161.0

[31336 rows x 4 columns]

In [9]:
new_df = pd.merge(df_grouped, df_grouped,  how='left', left_on=['location_id','partner_id'], right_on = ['partner_id','location_id'])

In [10]:
df

,location_id,partner_id,product_id,year,export_value,import_value,sitc_eci,sitc_coi,location_code,partner_code,sitc_product_code
945000,3,6,1352,2003.0,0.0,4281.0,1.535351,0.268432,AIA,ANT,8423
945001,25,6,1352,2003.0,0.0,26220.0,0.308971,-0.561269,BHS,ANT,8423
945002,29,6,1352,2003.0,0.0,84921.0,-0.546138,-0.787781,BLZ,ANT,8423
945003,32,6,1352,2003.0,8011.0,0.0,0.636739,1.422593,BRA,ANT,8423
945004,33,6,1352,2003.0,0.0,2387.0,0.499073,-0.065065,BRB,ANT,8423
...,...,...,...,...,...,...,...,...,...,...,...
3982467,166,242,1363,2019.0,2379.0,0.0,0.099708,0.079208,NZL,WSM,8443
3982468,127,210,1363,2019.0,0.0,2392.0,0.263772,-0.709830,LCA,SXM,8443
3982469,231,210,1363,2019.0,11067.0,0.0,1.528196,0.297825,USA,SXM,8443
3982470,77,27,1363,2019.0,1706.0,0.0,1.345057,1.075400,FRA,BLM,8443


In [ ]:
new_df = pd.merge(df_grouped, df_grouped,  how='left', left_on=['location_id','partner_id'], right_on = ['partner_id','location_id'])